In [1]:
import numpy as np
import requests 
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import io

dates = np.array(pd.date_range(start='06/01/2015',end='12/16/2019').date)

data_frames_renewable_prod = {}
data_frames_all_prod = {}

scrape_log = pd.DataFrame(columns=['date','exists','got data'])

for i in dates:
    date = str(i).replace('-','')
    if date in ['20170312','20160313']:
        continue
    
    url = 'http://content.caiso.com/green/renewrpt/{}_DailyRenewablesWatch.txt'.format(date)
    with requests.get(url, stream=True) as r:
#     browser = init_browser()
    
#     browser.visit(url)
#     time.sleep(2)
    
#     html = browser.html
#     browser.quit()
#     html = requests.get(url)
        data = bs(r.text, 'html.parser')

        text_data = data.text
        if '404 Not Found' in text_data:
            entry = pd.DataFrame({
                'date':[date],
                'exists':False,
                'got data':False
            })
            scrape_log = pd.concat([scrape_log, entry])

            continue 
        try:
            f = io.StringIO(text_data)
            top = pd.read_csv(f,delimiter='\t+', header=1, skipfooter=26)
            top['date'] = date

            f = io.StringIO(text_data)
            bottom = pd.read_csv(f,delimiter='\t+', skiprows=26, header=1)
            bottom['date'] = date

            data_frames_renewable_prod[date] = top
            data_frames_all_prod[date] = bottom
            entry = pd.DataFrame({
                'date':[date],
                'exists':True,
                'got data':True
            })
            scrape_log = pd.concat([scrape_log, entry])
        except:
            entry = pd.DataFrame({
                'date':[date],
                'exists':True,
                'got data':False
            })
            scrape_log = pd.concat([scrape_log, entry])


        
        
scrape_log[scrape_log.exists == False]

C:\Users\Scottinsacto\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
C:\Users\Scottinsacto\Anaconda3\lib\site-packages\ipykernel_launcher.py:49: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


,date,exists,got data
0,20190923,False,False
0,20190924,False,False
0,20190925,False,False
0,20190926,False,False
0,20190927,False,False
...,...,...,...
0,20191207,False,False
0,20191208,False,False
0,20191209,False,False
0,20191210,False,False


In [2]:
hourly_renewable_prod = pd.DataFrame([])
hourly_total_prod = pd.DataFrame([])
for i in dates:
    date = str(i).replace('-','')
    try:
        datar = data_frames_renewable_prod[date]
        datat = data_frames_all_prod[date]
    except KeyError:
        continue
    hourly_renewable_prod = pd.concat([hourly_renewable_prod, datar])
    hourly_total_prod = pd.concat([hourly_total_prod, datat])

In [3]:
pd.set_option('display.max_rows', 500)
hourly_renewable_prod['date'] = pd.to_datetime(hourly_renewable_prod['date'])
hourly_total_prod['date'] = pd.to_datetime(hourly_total_prod['date'])
print(hourly_renewable_prod.info())
print(hourly_total_prod.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37872 entries, 0 to 23
Data columns (total 9 columns):
Hour             37872 non-null int64
GEOTHERMAL       37872 non-null int64
BIOMASS          37872 non-null int64
BIOGAS           37872 non-null int64
SMALL HYDRO      37872 non-null int64
WIND TOTAL       37872 non-null int64
SOLAR PV         37872 non-null int64
SOLAR THERMAL    37872 non-null int64
date             37872 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(8)
memory usage: 2.9 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 37872 entries, 0 to 23
Data columns (total 7 columns):
Hour          37872 non-null int64
RENEWABLES    37872 non-null int64
NUCLEAR       37872 non-null int64
THERMAL       37872 non-null int64
IMPORTS       37872 non-null int64
HYDRO         37872 non-null int64
date          37872 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(6)
memory usage: 2.3 MB
None


In [4]:
hourly_total_prod['timestamp'] = hourly_total_prod['date'].astype(str) + 'T' + (hourly_total_prod['Hour']-1).astype(str) +':00:00'
hourly_total_prod['timestamp'] = pd.to_datetime(hourly_total_prod['timestamp'])
hourly_total_prod = hourly_total_prod[['timestamp','date','Hour','RENEWABLES','NUCLEAR','THERMAL','HYDRO']]

hourly_renewable_prod['timestamp'] = hourly_renewable_prod['date'].astype(str) + 'T' + (hourly_renewable_prod['Hour']-1).astype(str) +':00:00'
hourly_renewable_prod['timestamp'] = pd.to_datetime(hourly_renewable_prod['timestamp'])
hourly_renewable_prod['SOLAR TOTAL'] = hourly_renewable_prod['SOLAR PV'] + hourly_renewable_prod['SOLAR THERMAL']
hourly_renewable_prod['TOTAL'] = hourly_renewable_prod['SOLAR TOTAL'] + hourly_renewable_prod['WIND TOTAL'] +\
hourly_renewable_prod['GEOTHERMAL'] +hourly_renewable_prod['BIOMASS'] +hourly_renewable_prod['BIOGAS'] +hourly_renewable_prod['SMALL HYDRO']
hourly_renewable_prod = hourly_renewable_prod[['timestamp','date','Hour','TOTAL','SOLAR TOTAL','SOLAR PV','SOLAR THERMAL',
                                               'WIND TOTAL','GEOTHERMAL','BIOMASS','BIOGAS','SMALL HYDRO']]


display(hourly_renewable_prod.head())
display(hourly_total_prod.head())

,timestamp,date,Hour,TOTAL,SOLAR TOTAL,SOLAR PV,SOLAR THERMAL,WIND TOTAL,GEOTHERMAL,BIOMASS,BIOGAS,SMALL HYDRO
0,2015-06-01 00:00:00,2015-06-01,1,4669,0,0,0,2942,1038,321,209,159
1,2015-06-01 01:00:00,2015-06-01,2,4396,0,0,0,2708,1040,316,210,122
2,2015-06-01 02:00:00,2015-06-01,3,4197,0,0,0,2509,1039,319,209,121
3,2015-06-01 03:00:00,2015-06-01,4,4059,0,0,0,2372,1035,320,209,123
4,2015-06-01 04:00:00,2015-06-01,5,3694,0,0,0,1973,1035,317,208,161


,timestamp,date,Hour,RENEWABLES,NUCLEAR,THERMAL,HYDRO
0,2015-06-01 00:00:00,2015-06-01,1,4668,2280,7318,1050
1,2015-06-01 01:00:00,2015-06-01,2,4396,2277,7383,1077
2,2015-06-01 02:00:00,2015-06-01,3,4197,2274,6921,1066
3,2015-06-01 03:00:00,2015-06-01,4,4058,2275,6565,1072
4,2015-06-01 04:00:00,2015-06-01,5,3694,2276,7050,1159


In [5]:

from sqlalchemy import create_engine
from config import username, password

engine = create_engine('postgresql://{}:{}@localhost:5432/cali_renewable_db'.format(username,password))
con = engine.connect()

name_total = 'hourlytotal'
name_renew = 'hourlyrenewable'
schema = 'Production'
hourly_renewable_prod.to_sql(name=name_renew,con=con,schema=schema, if_exists='append',
                            index=False)

In [6]:
hourly_total_prod.to_sql(name=name_total, con=con, schema=schema,
                        if_exists='append', index=False)